In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import math
import seaborn as sns

In [6]:
games = pd.read_excel('Results.xlsx', sheet_name='Games')
turns = pd.read_excel('Results.xlsx', sheet_name='Turns')
pvp_games = games.query('GameState == "END" and idPerson1 not in [10, 30, 203, 217] and idPerson2 not in [10, 30, 203, 217]')
pvp_games_ids = pvp_games['idGame']

In [377]:
pvp_games_turns = []
pvp_games_states = []
for i in pvp_games_ids:
    pvp_game_i_turns = turns.query('Games_idGame == ' + str(i))
    for state in pvp_game_i_turns['State']:
        pvp_games_states.append(state)
    if max(pvp_game_i_turns['idTurn']) < 1000:
        pvp_games_turns.append(max(pvp_game_i_turns['idTurn']))

In [670]:
def digitalize_states(series):
    states = []
    for point in series:
        point = point[1:]
        point = point[:-1]
        point = point.split(",")
        x = int(point[0])
        y = int(point[1])
        pair = (x, y)
        states.append(pair)
    return states

In [671]:
def is_close(state1, state2):
    (x1, y1) = (state1[0], state1[1])
    (x2, y2) = state2
    if (abs(x1 - x2) + abs(y1 - y2) == 1):
        return True
    else:
        return False

In [688]:
def get_prob_matrix(N, pvp_games_states):
    states = digitalize_states(pvp_games_states)   
    frequencies = np.zeros((N**2, N**2))
    for i in range(1, len(states)):
        if (is_close(states[i], states[i-1])):
            frequencies[(N-2) * states[i-1][0] + states[i-1][1], (N-2) * states[i][0] + states[i][1]] +=1
    for i in range (0, frequencies.shape[0]):
        if (sum(frequencies[i]) != 0):
            frequencies[i] /= sum(frequencies[i])
    frequencies = frequencies[N:]
    frequencies = frequencies[:-N]
    frequencies = frequencies[:, N:]
    frequencies = frequencies[:, :-N]
    return frequencies

In [ ]:
def get_prob_matrix_symmetry(N, pvp_games_states):
    states = digitalize_states(pvp_games_states)   
    frequencies = np.zeros((N**2, N**2))

In [689]:
def get_fundamental_matrix(Q):
    return np.linalg.inv(np.eye(Q.shape[0]) - Q)

In [695]:
def get_mean_time(fundamental_matrix):
    c = np.ones(fundamental_matrix.shape[0])
    mean_times = fundamental_matrix @ c
    N = round(math.sqrt(fundamental_matrix.shape[0]))
    return mean_times[N * (N // 2) + N // 2]

In [697]:
N = 16

probability_matrix = get_prob_matrix(N, pvp_games_states)
fundamental_matrix = get_fundamental_matrix(probability_matrix)
mean_time = get_mean_time(fundamental_matrix)
print(mean_time)

119.25619548594251


In [694]:
with open("q.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(probability_matrix)
    
with open("fundamental.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(fundamental_matrix)
